# Tutorial: VisionKG - A Data-Centric Way to Train your own Obejct Detection Models

# 1. QuickView of VisionKG

## 1.1 One-Click to meet VisionKG

In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import output
# install our vision api
!python -m pip install git+https://github.com/cqels/vision.git --force
output.clear()

## 1.2 Query a Dataset as YOU need via VisionKG

In [ ]:
# import SemkgAPI
from vision_utils import semkg_api, data
from skimage import io
import matplotlib.pyplot as plt
from torch_model_zoo import utils

query_string='''# Give me 100 images contain Car and Pedestrian with its annotations.
PREFIX cv:<http://vision.semkg.org/onto/v0.1/>
PREFIX schema:<http://schema.org/>
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
SELECT ?datasetName (STR(?_imageName) AS ?imageName) ?imageUrl (xsd:integer(?_imageWidth) AS ?imageWidth) (xsd:integer(?_imageHeight) AS ?imageHeight) ?labelName ?bbCentreX ?bbCentreY ?bbWidth ?bbHeight
WHERE {
  {
    SELECT ?image
    WHERE{
      ?image cv:hasAnnotation ?annotation1.
      ?annotation1 a cv:ObjectDetectionAnnotation.
      ?annotation1 cv:hasLabel ?label1.
      ?label1 cv:label "car".

      ?image cv:hasAnnotation ?annotation2.
      ?annotation2 a cv:ObjectDetectionAnnotation.
      ?annotation2 cv:hasLabel ?label2.
      ?label2 cv:label "person".

      ?image schema:isPartOf / schema:name ?datasetName .
      FILTER regex(?datasetName, "coco2017", "i")
    }
    GROUP BY ?image
    LIMIT 100
  }
  ?image schema:isPartOf / schema:name ?datasetName .
  ?image schema:name ?_imageName.
  OPTIONAL{?image schema:contentUrl ?imageUrl}.
  ?image cv:hasAnnotation ?annotation.
  ?image cv:imgWidth ?_imageWidth.
  ?image cv:imgHeight ?_imageHeight.
  ?annotation cv:hasLabel/cv:label ?labelName.
  ?annotation cv:hasBox ?bbox.
  ?bbox cv:boxHeight ?bbHeight.
  ?bbox cv:boxWidth ?bbWidth.
  ?bbox cv:centerX ?bbCentreX.
  ?bbox cv:centerY ?bbCentreY.
}
'''

#Query and return result
result=semkg_api.query(query_string)
result = utils.visionkg2cocoDet(result)

# #Display sample images
# rows=3
# cols=4
# f, ax_arr = plt.subplots(rows, cols, figsize=(16,8))
# for j, row in enumerate(ax_arr):
#     for i, ax in enumerate(row):
#         if j*cols+i < len(result['images']):
#             image = io.imread(semkg_api.SEMKG_IMAGES_HOST + result['images'][j*cols+i]['image_path'])
#             ax.imshow(image)

# f.suptitle("Sample images from the query result", fontsize=16)
# plt.show()

# 2. Object Detection in Practice starting from VisionKG

##2.1 VisionKG meet mmdetection

In [ ]:
# clone the toolbox for object detection
!git clone https://github.com/open-mmlab/mmdetection.git

# install dependencies
%cd mmdetection/
!python -m pip install -r requirements.txt
!python -m pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html
!python setup.py develop

output.clear()

## 2.2 Prepare and set parameters for training and evaluation

In [ ]:
from os.path import join as opj
from shutil import copy
from torch_model_zoo import utils
from mmdetection_configs import configs_fcos_visionKG

path_to_anno_mixedDatasets = opj('data/mixedDatasets/','test_query_api_image.json')
filter_cat_nms = ['person', 'car']
params = utils.prepare_for_training(path_to_anno_mixedDatasets, result, filter_cat_nms)
path_to_config = 'configs/fcos/fcos_visionKG.py'
path_to_work_dir = 'mixedDatasets/logs_visionKG/'
copy(configs_fcos_visionKG.__file__, path_to_config)
nms_categories = params['CAT_NMS']
num_categories = len(nms_categories)

##2.3 Data-Playground

In [ ]:
if num_categories > 4:
  cat_nms = nms_categories[0:4]
else:
  cat_nms = nms_categories
utils.show_annotation(path_to_anno_mixedDatasets, cat_nms, show_num=6)
utils.show_cat_distribution(path_to_anno_mixedDatasets, cat_nms)

## 2.4 Perform Training & Evaluation on your chosen Object Detection tool

In [ ]:
# Training based on the queried MixedDataset
# For more params-setting, please check:
# https://mmdetection.readthedocs.io/en/latest/
%run tools/train.py {path_to_config} \
--cfg-options model.bbox_head.num_classes={num_categories} \
train_dataloader.dataset.metainfo.classes="$nms_categories" val_dataloader.dataset.metainfo.classes="$nms_categories"